# Import

In [354]:
data_path='data/'
api_key='ehlVSIKSMiugJZHlii3sU7OKCexe4MIlnYenGric'
import pandas as pd 
import numpy as np

# Analysis

## Take  a look on dataset

In [355]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(f"{data_path}epi_r.csv")
for name, count in df.nunique().items():
    print(f"{name}: {count}")

title: 17736
rating: 8
calories: 1858
protein: 282
fat: 326
sodium: 2434
#cakeweek: 2
#wasteless: 2
22-minute meals: 2
3-ingredient recipes: 2
30 days of groceries: 2
advance prep required: 2
alabama: 2
alaska: 2
alcoholic: 2
almond: 2
amaretto: 2
anchovy: 2
anise: 2
anniversary: 2
anthony bourdain: 2
aperitif: 2
appetizer: 2
apple: 2
apple juice: 2
apricot: 2
arizona: 2
artichoke: 2
arugula: 2
asian pear: 2
asparagus: 2
aspen: 2
atlanta: 2
australia: 2
avocado: 2
back to school: 2
backyard bbq: 2
bacon: 2
bake: 2
banana: 2
barley: 2
basil: 2
bass: 2
bastille day: 2
bean: 2
beef: 2
beef rib: 2
beef shank: 2
beef tenderloin: 2
beer: 2
beet: 2
bell pepper: 2
berry: 2
beverly hills: 2
birthday: 2
biscuit: 2
bitters: 2
blackberry: 2
blender: 2
blue cheese: 2
blueberry: 2
boil: 2
bok choy: 2
bon appétit: 2
bon app��tit: 2
boston: 2
bourbon: 2
braise: 2
bran: 2
brandy: 2
bread: 2
breadcrumbs: 2
breakfast: 2
brie: 2
brine: 2
brisket: 2
broccoli: 2
broccoli rabe: 2
broil: 2
brooklyn: 2
brown r

The dataset have target feature:
```rating```.

There is also features: ```title, calories, protein, fat, sodium```



Dataset have 2 categorical features after the One-Hot Encoding conversion.

first categorical feature is:

Ingredients: ```almond, anchovy, apple, avocado, bacon```

second categorical feature is tags

The tags may contain:
* Recipe characteristics: ```#cakeweek, #wasteless, 22-minute meals, 3-ingredient recipes...```

* Geographical affiliation: ```alabama, arizona, boston, france, italy...```

* Dietary restrictions and features: ```dairy free, gluten free, vegan...```

* Cooking methods: ```bake, boil, grill, slow cooker...```

* Holidays and events: ```christmas, halloween, valentine's day...```

* Machinery and tools: ```blender, food processor, pressure cooker, microwave...```

## Let's sort columns in the dataset, and get nutrients for ingredients

### this is list of nutrients, what we need

In [356]:
nutrients_list = [
    # Основные макронутриенты 
    "Total lipid (fat)",
    "Fatty acids, total saturated",
    "Fatty acids, total monounsaturated",
    "Fatty acids, total polyunsaturated",
    "Fatty acids, total trans",
    "Cholesterol",
    "Carbohydrate, by difference",
    "Sodium, Na", 
    "Fiber, total dietary",
    "Protein",
    "Sugars, Total",  
    "Energy", 
    "Vitamin A, RAE",  
    "Vitamin C, total ascorbic acid", 
    "Vitamin D (D2 + D3)",
    "Vitamin E (alpha-tocopherol)",
    "Vitamin K (phylloquinone)",
    "Thiamin",
    "Riboflavin",
    "Niacin",
    "Vitamin B-6",
    "Folate, total",
    "Vitamin B-12",
    
    # Минералы
    "Calcium, Ca",
    "Iron, Fe",
    "Phosphorus, P",
    "Magnesium, Mg",
    "Zinc, Zn",
    "Copper, Cu",
    "Manganese, Mn",
    "Selenium, Se",
    "Potassium, K",
    
    # Специфичные компоненты 
    "Ash",
    "Nitrogen",
    "Water",
    
    # Разные формы сахаров
    "Fructose",
    "Glucose",
    "Sucrose",
    "Galactose"
]


### Funcs for primary sort ingredients and get cache for nutrients 

In [357]:
import requests
import json
def load_nutrients_cache(path_to_file):
    try:
        with open(path_to_file, 'r', encoding='utf-8') as f:
            try:
                return json.load(f)
            except:
                return {}
    except:
        return {}
def save_to_cache(path_to_file, ingredient, nutrient_dict, cache):
    cache[ingredient]=nutrient_dict
    with open(path_to_file, 'w', encoding='utf-8') as f:
        json.dump(cache, f, indent=4, ensure_ascii=False)

def get_nutrients_info(ingredient):
    ingredient_info=None
    params = {
        "query": f'"{ingredient}"',
        "api_key": api_key,
        "dataType": ["Foundation"]  
    }
    cache=load_nutrients_cache(f"{data_path}ingridient_nutr_cache.json")
    non_ingredient_cache=load_nutrients_cache(f"{data_path}not_ingridient_cache.json")
    if ingredient in cache:
        return cache[ingredient]
    elif ingredient not in non_ingredient_cache:
        response = requests.get('https://api.nal.usda.gov/fdc/v1/foods/search', params=params)
        if response.status_code == 200:
            ingredient_info = response.json().get('foods')
            if ingredient_info and ingredient_info[0].get('score') < 200:
                ingredient_info=None
            if ingredient_info:
                ingredient_info=ingredient_info[0].get('foodNutrients')
                nutrient_df=pd.json_normalize(ingredient_info)[['nutrientName','unitName','value']]
                nutrient_dict=nutrient_df[nutrient_df['nutrientName'].isin(nutrients_list)].to_dict()       
                save_to_cache(f"{data_path}ingridient_nutr_cache.json",ingredient, nutrient_dict, cache)
    if not ingredient_info:
        save_to_cache(f"{data_path}not_ingridient_cache.json",ingredient, ingredient_info, non_ingredient_cache)
    return ingredient_info
def is_ingredient(ingredient):
    data=get_nutrients_info(ingredient)
    return True if data else False


In [358]:
features =[]
for column in df.columns:
    for ingredients in column.split("/"):
        if is_ingredient(ingredients):
            features.append(column)
            break
print("a list of possible ingredients")
print (features)


a list of possible ingredients
['fat', 'sodium', 'almond', 'apple', 'apple juice', 'apricot', 'arugula', 'asparagus', 'avocado', 'bacon', 'banana', 'barley', 'bean', 'beef', 'beef tenderloin', 'beet', 'blackberry', 'blueberry', 'boil', 'bok choy', 'braise', 'bran', 'bread', 'breakfast', 'broccoli', 'broil', 'brussel sprout', 'bulgur', 'butter', 'buttermilk', 'cabbage', 'cantaloupe', 'carrot', 'cashew', 'cauliflower', 'celery', 'cheddar', 'cherry', 'chestnut', 'chicken', 'chickpea', 'cobbler/crumble', 'coconut', 'cod', 'cookie', 'cookies', 'corn', 'cottage cheese', 'crab', 'cranberry', 'cream cheese', 'cucumber', 'dill', 'egg', 'eggplant', 'fat free', 'feta', 'fig', 'fish', 'garlic', 'grains', 'grape', 'grapefruit', 'green onion/scallion', 'ham', 'hazelnut', 'hominy/cornmeal/masa', 'honeydew', 'hummus', 'kale', 'kiwi', 'kosher', 'lamb', 'leek', 'lentil', 'lettuce', 'low fat', 'low/no sugar', 'macadamia nut', 'mango', 'meat', 'melon', 'milk/cream', 'monterey jack', 'mozzarella', 'mushroo

In [359]:
non_ingreedients = []
for column in df.columns:
    for non_ingreedient in column.split("/"):
        if not is_ingredient(non_ingreedient):
            non_ingreedients.append(column)
            break
print("a list of possible non ingredients")
print (non_ingreedients)


a list of possible non ingredients
['title', 'rating', 'calories', 'protein', '#cakeweek', '#wasteless', '22-minute meals', '3-ingredient recipes', '30 days of groceries', 'advance prep required', 'alabama', 'alaska', 'alcoholic', 'amaretto', 'anchovy', 'anise', 'anniversary', 'anthony bourdain', 'aperitif', 'appetizer', 'arizona', 'artichoke', 'asian pear', 'aspen', 'atlanta', 'australia', 'back to school', 'backyard bbq', 'bake', 'basil', 'bass', 'bastille day', 'beef rib', 'beef shank', 'beer', 'bell pepper', 'berry', 'beverly hills', 'birthday', 'biscuit', 'bitters', 'blender', 'blue cheese', 'bon appétit', 'bon app��tit', 'boston', 'bourbon', 'brandy', 'breadcrumbs', 'brie', 'brine', 'brisket', 'broccoli rabe', 'brooklyn', 'brown rice', 'brownie', 'brunch', 'buffalo', 'buffet', 'bulgaria', 'burrito', 'butternut squash', 'butterscotch/caramel', 'cake', 'california', 'calvados', 'cambridge', 'campari', 'camping', 'canada', 'candy', 'candy thermometer', 'capers', 'caraway', 'cardamom

This API method doesn't work very well, so we'll sort the features manually.

remove non_ingredients from features

In [360]:
non_ingredients = [
    'fat',               
    'sodium',            
    'boil',              
    'braise',            
    'broil',            
    'breakfast',        
    'cobbler/crumble',  
    'cookie',           
    'cookies',        
    'fat free',        
    'kosher',      
    'low fat',          
    'low/no sugar',    
    'no sugar added',   
    'no-cook',     
    'raw',             
    'stuffing/dressing',
    'summer',       
    'winter',         
    'wheat/gluten-free'
]
for value in non_ingredients:
    if value in features:
        if value not in df.columns:
            print(f"problem with: {value}")
        features.remove(value)

add actually ingredients from dataset to features

In [361]:
actually_ingredients=actually_ingredients = [
    'anchovy', 'anise', 'apple juice', 'artichoke',
    'asian pear', 'basil', 'bass', 'beef rib',
    'beef shank', 'beer', 'bell pepper', 'berry',
    'bitters', 'blue cheese', 'brie', 'brisket',
    'broccoli rabe', 'brown rice', 'brownie', 'butternut squash',
    'butterscotch/caramel', 'cake', 'calvados', 'capers',
    'caraway', 'cardamom', 'caviar', 'chard',
    'cheese', 'chile', 'chile pepper', 'chili',
    'chive', 'chocolate', 'cilantro', 'cinnamon',
    'citrus', 'clam', 'clove', 'coffee',
    'collard greens', 'cornmeal', 'couscous', 'cranberry sauce',
    'crêpe', 'currant', 'curry', 'custard',
    'date', 'dried fruit', 'duck', 'egg nog',
    'endive', 'escarole', 'fennel', 'flat bread',
    'fontina', 'fruit juice', 'ginger', 'goat cheese',
    'goose', 'gouda', 'granola', 'green bean',
    'ground beef', 'ground lamb', 'guava', 'halibut',
    'hamburger', 'herb', 'hominy/cornmeal/masa', 'honey',
    'horseradish', 'ice cream', 'iced coffee', 'iced tea',
    'jalapeño', 'jam or jelly', 'jerusalem artichoke', 'jícama',
    'kumquat', 'lamb chop', 'lamb shank', 'lasagna',
    'leafy green', 'legume', 'lemon', 'lemon juice',
    'lemongrass', 'lima bean', 'lime', 'lime juice',
    'lingonberry', 'lobster', 'macaroni and cheese', 'maple syrup',
    'mayonnaise', 'meatball', 'meatloaf', 'milk/cream',
    'mint', 'molasses', 'muffin', 'mussel',
    'mustard greens', 'noodle', 'nutmeg', 'octopus',
    'okra', 'omelet', 'oregano', 'orzo',
    'papaya', 'paprika', 'parsley', 'parsnip',
    'passion fruit', 'persimmon', 'phyllo/puff pastry dough', 'poblano',
    'pomegranate juice', 'poppy', 'pork rib', 'pork tenderloin',
    'pot pie', 'potato salad', 'prosciutto', 'quail',
    'quiche', 'quince', 'rabbit', 'rack of lamb',
    'radicchio', 'radish', 'rhubarb', 'rosemary',
    'rosé', 'rum', 'saffron', 'sage',
    'sake', 'salad dressing', 'sardine', 'scallop',
    'scotch', 'sesame oil', 'shellfish', 'sherry',
    'snapper', 'sorbet', 'soufflé/meringue', 'soup/stew',
    'sour cream', 'sourdough', 'soy sauce', 'spice',
    'squid', 'stock', 'stuffing/dressing', 'sugar snap pea',
    'sweet potato/yam', 'swiss cheese', 'swordfish', 'tamarind',
    'tangerine', 'tapioca', 'tarragon', 'tea',
    'tofu', 'tortillas', 'tree nut', 'triple sec',
    'tropical fruit', 'trout', 'turnip', 'vanilla',
    'veal', 'venison', 'vermouth', 'vinegar',
    'waffle', 'wasabi', 'watercress', 'watermelon',
    'white wine', 'wine', 'yellow squash', 'yuca'
]
for value in actually_ingredients:
    if value not in features:
        if value not in df.columns:
            print(f"problem with: {value}")
        features.append(value)


## lets sort columns in our dataset


In [362]:
X=df[features]
y=df['rating']

In [363]:
print(X.shape) 
print(y.shape) 

X.sum().sort_values(ascending=False)

(20052, 322)
(20052,)


onion         2238.0
tomato        2140.0
milk/cream    1995.0
egg           1768.0
herb          1681.0
               ...  
waffle           1.0
crêpe            1.0
quiche           1.0
sorbet           1.0
sourdough        1.0
Length: 322, dtype: float64

## Let's start tuning the hyperparameters

это чисто препроцессинг, то что я сделал выше мб оформлю его так же шобы в пайплайн запихать 

In [364]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

#### Train-test-validation split 

In [365]:
# def rating_to_сat(y):
#     bins = [0, 2, 4, 6]
#     labels = ['bad', 'ok', 'good']
#     return pd.cut(y, bins=bins, labels=labels)

class TrainValidationTest(BaseEstimator, TransformerMixin):
        def __init__(self, test_size=0.2, random_state=21, stratified=True):
            self.test_size = test_size
            self.random_state = random_state
            self.stratified = stratified
        def split(self, X , y):
            stratify = y if self.stratified else None
            X_train, X_test, y_train, y_test =train_test_split(
                X, y, test_size=self.test_size, random_state=self.random_state, stratify=stratify
            )
            X_train, X_valid, y_train, y_valid =train_test_split(
                X_train, y_train, test_size=self.test_size/(1-self.test_size), random_state=self.random_state, stratify=stratify
            )
            return X_train, X_valid, X_test, y_train, y_valid, y_test

#### Model selection class


In [366]:
from copy import deepcopy
from itertools import product
from sklearn.metrics import get_scorer
def manual_grid_search(X_train, y_train, X_valid, y_valid, model_estimator,param_grid, cv, scoring='accuracy_score',  n_jobs=-1, error_score=np.nan):
    res=[]
    estimator=deepcopy(model_estimator)
    total_combinations = np.prod([len(v) for v in param_grid.values()])
    # get scorer for valid score
    scorer = get_scorer(scoring)

    for params in tqdm(list(product(*param_grid.values())), total=total_combinations, desc="GridSearch"):

        current_model = deepcopy(estimator)
        current_model.set_params(**(dict(zip(param_grid.keys(), params))))
        try:
            score=cross_val_score(
                current_model, X_train, y_train, cv=cv, scoring=scoring, n_jobs=n_jobs,error_score=error_score
            )

            current_model.fit(X_train, y_train)
            valid_score=scorer(current_model, X_valid, y_valid)

            res.append({
                'train_score': score.mean(),
                'valid_score': valid_score,
                'params': dict(zip(param_grid.keys(), params))
            })


        except Exception as e:
            print(f"Error with params {params}: {str(e)}")
            res.append({
                **(dict(zip(param_grid.keys(), params))),
                'train_score': 'Na',
                'valid_score': 'Na'
            })
            continue
        
    return pd.DataFrame(res).sort_values('valid_score', ascending=False)

In [367]:
class ModelSelection:
    def __init__ (self,estimators,estimators_name, grid_dict, scoring='accuracy_score'):
        self.estimators=estimators
        self.scoring=scoring
        self.estimators_name=estimators_name
        self.grid_dict=grid_dict
        self.results=pd.DataFrame()
        self.best_res=[]
    def choose(self,X_train, y_train, X_valid, y_valid, cv=2):
        """
        Method choose() takes X_train, y_train, X_valid, y_valid, optional cv, 
        and returns the name of the best model among all the models on the validation set
        """
        for estimator in self.estimators:
            model_name= self.estimators_name[self.estimators.index(estimator)]
            search_res=manual_grid_search(X_train,y_train,X_valid, y_valid,estimator,self.grid_dict[model_name], cv=cv, scoring=self.scoring)
            search_res['model']=model_name
            best_params = search_res.head(1)['params'].item()
            best_train_score = search_res.head(1)['train_score'].item()
            best_valid_score = search_res.head(1)['valid_score'].item()
            self.results=pd.concat([self.results, search_res], axis=0, ignore_index=True)
            self.best_res.append({
                'model': model_name,
                'params': best_params,
                'valid_score': best_valid_score,
                'train_score': best_train_score
            })
            
            print(f"Best params for model{model_name}: {best_params}")
            print(f"training accuracy: {best_train_score:.3f}")
            print(f"Best on validation set accuracy score: {best_valid_score:.3f}")
        self.best_res=pd.DataFrame(self.best_res)
        self.best_res.sort_values('valid_score', ascending=False,inplace=True)
        best_model_name=self.best_res.head(1)['model'].item()
        print(f"\nBest model: {best_model_name}\nwith params: {self.best_res.head(1)['params'].item()}\nand score: {self.best_res.head(1)['valid_score'].item()}")
        return self.results.sort_values('valid_score', ascending=False)
    # def show_results(self):
        # return self.results.groupby(by="model").sort_values('valid_score', ascending=False)
    def fit_score(self,X_comb,y_comb, X_test, y_test):
        params=self.best_res.head(1)['params'].item()
        model_name=self.best_res.head(1)['model'].item()
        print(f'Taking model: {model_name} with params:{params}')
        best_estimator=self.estimators[self.estimators_name.index(model_name)]
        best_estimator.set_params(**(params))
        best_estimator.fit(X_comb,y_comb)
        scorer=get_scorer(self.scoring)
        print("the final score is :",(scorer(best_estimator,X_test,y_test)))        

In [368]:
def select_parametrs(X,y,estimators,estimators_name, grid_dict, scoring='accuracy_score'):
    splitter = TrainValidationTest(stratified=False)
    X_train, X_valid, X_test, y_train, y_valid, y_test = splitter.split(X, y)
    search_parametrs= ModelSelection(estimators,estimators_name, grid_dict, scoring)
    result_serching_df=search_parametrs.choose(X_train, y_train, X_valid, y_valid)
    X_comb=pd.concat([X_train, X_valid], axis=0, ignore_index=True)
    y_comb=pd.concat([y_train, y_valid], axis=0, ignore_index=True)
    print("Calc accuracy on the best model")
    search_parametrs.fit_score(X_comb,y_comb, X_test,y_test)
    return(result_serching_df)

# Regression serch params

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge

estimators = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    RandomForestRegressor(random_state=21),
    GradientBoostingRegressor(random_state=21),
    SVR(),
    KNeighborsRegressor(),
    KernelRidge(),
]

estimators_name = [
    'LinearRegression',
    'Ridge',
    'Lasso',
    'ElasticNet',
    'RandomForest',
    'GradientBoosting',
    'SVR',
    'KNN',
    'KernelRidge',
]

grid_dict = {
    'LinearRegression': {
        'fit_intercept': [True, False],
        'copy_X': [True, False],
        'positive': [True, False],
        'n_jobs': [-1]
    },
    'Ridge': {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
        'fit_intercept': [True, False],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    },
    'Lasso': {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
        'fit_intercept': [True, False],
        'selection': ['cyclic', 'random'],
        'positive': [True, False]
    },
    'ElasticNet': {
        'alpha': [0.001, 0.01, 0.1, 1.0],
        'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
        'fit_intercept': [True, False],
        'selection': ['cyclic', 'random']
    },
    'RandomForest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 5, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', None],
        'bootstrap': [True, False]
    },
    'GradientBoosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.001, 0.01, 0.1],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'max_features': ['sqrt', 'log2'],
        'subsample': [0.8, 1.0]
    },
    'SVR': {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'C': [0.1, 1.0, 10.0],
        'epsilon': [0.01, 0.1, 0.5],
        'gamma': ['scale', 'auto']
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'leaf_size': [10, 30, 50],
        'p': [1, 2]
    },
    'KernelRidge': {
        'alpha': [0.1, 1.0, 10.0],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'gamma': [None, 0.1, 1.0],
        'degree': [2, 3, 4]
    },
    'XGBoost': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'gamma': [0, 0.1, 0.2],
        'reg_alpha': [0, 0.1, 1],
        'reg_lambda': [0, 0.1, 1]
    },
    'LightGBM': {
        'num_leaves': [15, 31, 63],
        'max_depth': [-1, 5, 10],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [50, 100, 200],
        'min_child_samples': [5, 10, 20],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'reg_alpha': [0, 0.1, 1],
        'reg_lambda': [0, 0.1, 1]
    },
    'CatBoost': {
        'iterations': [50, 100, 200],
        'depth': [4, 6, 8],
        'learning_rate': [0.01, 0.05, 0.1],
        'l2_leaf_reg': [1, 3, 5],
        'border_count': [32, 64, 128],
        'subsample': [0.6, 0.8, 1.0]
    }
}

In [370]:
select_parametrs(X,y,estimators,estimators_name, grid_dict, scoring='neg_root_mean_squared_error')

GridSearch:   0%|          | 0/8 [00:00<?, ?it/s]

Best params for modelLinearRegression: {'fit_intercept': True, 'copy_X': True, 'positive': False, 'n_jobs': -1}
training accuracy: -1.339
Best on validation set accuracy score: -1.292


GridSearch:   0%|          | 0/84 [00:00<?, ?it/s]

/home/chumba/projects/DS_Bootcamp.Team01.ID_886525-Team_TL_shirledo.396ad497_d1b1_4d29-1/src/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/chumba/projects/DS_Bootcamp.Team01.ID_886525-Team_TL_shirledo.396ad497_d1b1_4d29-1/src/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/chumba/projects/DS_Bootcamp.Team01.ID_886525-Team_TL_shirledo.396ad497_d1b1_4d29-1/src/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/chumba/projects/DS_Bootcamp.Team01.ID_886525-Team_TL_shirledo.396ad497_d1b1_4d29-1/src/venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which me

Best params for modelRidge: {'alpha': 100.0, 'fit_intercept': True, 'solver': 'saga'}
training accuracy: -1.325
Best on validation set accuracy score: -1.308


...

Best params for modelKNN: {'n_neighbors': 9, 'weights': 'distance', 'algorithm': 'kd_tree', 'leaf_size': 30, 'p': 2}
training accuracy: -1.427
Best on validation set accuracy score: -1.342


GridSearch:   0%|          | 0/108 [00:00<?, ?it/s]

/home/chumba/projects/DS_Bootcamp.Team01.ID_886525-Team_TL_shirledo.396ad497_d1b1_4d29-1/src/venv/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/chumba/projects/DS_Bootcamp.Team01.ID_886525-Team_TL_shirledo.396ad497_d1b1_4d29-1/src/venv/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/chumba/projects/DS_Bootcamp.Team01.ID_886525-Team_TL_shirledo.396ad497_d1b1_4d29-1/src/venv/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/home/chumba/projects/DS_Bootcamp.Team01.ID_886525-Team_TL_shirledo.396ad497_d1b1_4d29-1/src/venv/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning: Singular

Best params for modelKernelRidge: {'alpha': 10.0, 'kernel': 'poly', 'gamma': 0.1, 'degree': 4}
training accuracy: -1.315
Best on validation set accuracy score: -1.300

Best model: RandomForest
with params: {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': True}
and score: -1.2655243055301424
Calc accuracy on the best model
Taking model: RandomForest with params:{'n_estimators': 50, 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': True}
the final score is : -1.287821723812737


,train_score,valid_score,params,model
212,-1.301510,-1.265524,"{'n_estimators': 50, 'max_depth': None, 'min_s...",RandomForest
213,-1.299593,-1.265561,"{'n_estimators': 100, 'max_depth': None, 'min_...",RandomForest
214,-1.299339,-1.265718,"{'n_estimators': 200, 'max_depth': None, 'min_...",RandomForest
215,-1.306118,-1.265953,"{'n_estimators': 100, 'max_depth': None, 'min_...",RandomForest
216,-1.306251,-1.266002,"{'n_estimators': 200, 'max_depth': None, 'min_...",RandomForest
...,...,...,...,...
1659,-131.683225,-59.327460,"{'alpha': 1.0, 'kernel': 'sigmoid', 'gamma': 1...",KernelRidge
1660,-131.683225,-59.327460,"{'alpha': 1.0, 'kernel': 'sigmoid', 'gamma': 1...",KernelRidge
1661,-90.714581,-171.905865,"{'alpha': 0.1, 'kernel': 'sigmoid', 'gamma': 1...",KernelRidge
1663,-90.714581,-171.905865,"{'alpha': 0.1, 'kernel': 'sigmoid', 'gamma': 1...",KernelRidge
